In [8]:
from pvsamlab.models import PVSystem, SolarResource, SystemDesign, Losses, Module, Inverter

# Step 1: Create a PVSystem Instance with Custom Parameters
pv_system = PVSystem(
    solar_resource=SolarResource(
        albedo=0.18,  # Monthly albedo value
        irrad_mode=1,  # Perez model
        sky_model=3
    ),
    system_design=SystemDesign(
        system_capacity=50000.0,  # 50 MW DC system
        inverter_count=25,  # 25 inverters
        subarray1_gcr=0.28
    ),
    losses=Losses(
        acwiring_loss=1.2,
        subarray1_dcwiring_loss=2.5,
        subarray1_soiling=4.0  # Monthly soiling loss
    ),
    module=Module(
        v_oc=65.0,
        i_sc=6.2,
        v_mp=55.0,
        i_mp=5.8,
        area=1.64
    ),
    inverter=Inverter(
        paco=4000.0,  # 4 kW max AC power
        efficiency=97.5
    )
)

# Step 2: Retrieve and Print Initial Outputs
print("✅ Initial Simulation Results:")
print(f"📊 Annual Energy: {pv_system.outputs['mwh_per_year']} MWh")
print(f"⚡ Capacity Factor: {pv_system.outputs['mwh_per_year'] / (pv_system.system_capacity * 8.76) * 100:.2f}%")
print(f"🔋 Inverter Efficiency Loss: {pv_system.outputs['energy_losses_summary'].get('inverter_efficiency_loss', 'N/A')}%")

# Step 3: Dynamically Update a Parameter Using `update_parameter()`
print("\n🔄 Updating Inverter Count...")
pv_system.update_parameter("system_design", "inverter_count", 30)

# Step 4: Retrieve and Print Updated Outputs
print("✅ Updated Simulation Results:")
print(f"📊 Annual Energy: {pv_system.outputs['mwh_per_year']} MWh")
print(f"⚡ Capacity Factor: {pv_system.outputs['mwh_per_year'] / (pv_system.system_capacity * 8.76) * 100:.2f}%")
print(f"🔋 Inverter Efficiency Loss: {pv_system.outputs['energy_losses_summary'].get('inverter_efficiency_loss', 'N/A')}%")


Exception: albedo error: Pvsamv1 error accessing variable albedo. Either parameter does not exist or is not numeric type.

In [3]:
pv_system

NameError: name 'pv_system' is not defined

In [7]:
Module(
        model_type=2,  # 6-parameter model
        v_oc=65.0,
        i_sc=6.2,
        v_mp=55.0,
        i_mp=5.8,
        area=1.64
    )

Module(model_type=2, v_oc=65.0, i_sc=6.2, v_mp=55.0, i_mp=5.8, area=1.64, n_series=96, t_noct=46.0, standoff=6.0, mounting=0, is_bifacial=False, bifacial_transmission_factor=0.013, bifaciality=0.7)

In [1]:
from pvsamlab.models import Location, PVSystem, Module, Inverter, Losses, SystemDesign, SolarResource

# Create a PVSystem instance with default parameters

print(Location())

{
    "lat": 0.0,
    "lon": 0.0,
    "elev": 0.0,
    "tz": 0
}


In [2]:
Module()

2025-03-07 18:16:33,342 - ERROR - ❌ PAN file not found: /Users/ihorkoshkin/Library/Mobile Documents/com~apple~CloudDocs/Documents/jupyter/pvsamlab/pvsamlab/data/modules/JAM66D45-640LB(3.2+2.0mm).PAN


{
    "module_model": 2,
    "params": {}
}

In [4]:
Inverter()

2025-03-07 18:16:59,530 - ERROR - ❌ OND file not found: /Users/ihorkoshkin/Library/Mobile Documents/com~apple~CloudDocs/Documents/jupyter/pvsamlab/pvsamlab/data/inverters/Sungrow_SG4400UD-MV-US_20230817_V14_PVsyst.6.8.6（New Version).OND


{
    "inverter_model": 1,
    "params": {}
}

In [5]:
SolarResource()

{
    "use_wf_albedo": 0,
    "irrad_mode": 0,
    "sky_model": 2,
    "solar_resource_file": "data/weather_files/default_weather.csv",
    "solar_resource_data": null
}

In [6]:
SystemDesign()

{
    "kwac": 100000,
    "target_dcac_ratio": 1.35,
    "system_voltage": 1500,
    "system_capacity": 135000.0,
    "inverter_count": null,
    "subarray1_nstrings": null,
    "subarray1_modules_per_string": null,
    "subarray1_track_mode": 1,
    "subarray1_tilt": 0.0,
    "subarray1_azimuth": 180.0,
    "subarray1_backtrack": 1,
    "subarray1_gcr": 0.33,
    "mppt_low_inverter": 250.0,
    "mppt_hi_inverter": 800.0,
    "dcac_ratio": null
}

In [7]:
Losses()

{
    "acwiring_loss": 1.0,
    "subarray1_dcwiring_loss": 2.0,
    "subarray1_soiling": 5.0
}

In [8]:
PVSystem()

2025-03-07 18:17:57,752 - ERROR - ❌ PAN file not found: /Users/ihorkoshkin/Library/Mobile Documents/com~apple~CloudDocs/Documents/jupyter/pvsamlab/pvsamlab/data/modules/JAM66D45-640LB(3.2+2.0mm).PAN
2025-03-07 18:17:57,754 - ERROR - ❌ OND file not found: /Users/ihorkoshkin/Library/Mobile Documents/com~apple~CloudDocs/Documents/jupyter/pvsamlab/pvsamlab/data/inverters/Sungrow_SG4400UD-MV-US_20230817_V14_PVsyst.6.8.6（New Version).OND


Exception: subarray1_nstrings error: subarray1_nstrings assignment value must be numeric, string, tuple or dict.